In [5]:
#import os

In [6]:
#pwd

In [7]:
#os.chdir("../")

In [8]:
#%pwd

In [9]:
## Create the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
     root_dir:Path
     data_path:Path
     model_ckpt:Path
     num_train_epochs:int
     warmup_steps:int
     per_device_train_batch_size:int
     weight_decay:float
     logging_steps:int
     evaluation_strategy:str
     eval_steps:int
     save_steps:float
     gradient_accumulation_steps:int


In [10]:
## Create the configuration Manager module:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments

        create_directories([config.root_dir])


        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            #per_device_eval_batch_size=params.per_device_eval_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.evaluation_strategy,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )

        return model_trainer_config

In [11]:
## Create component for the model trainer
from transformers import TrainingArguments,Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
from datasets import load_from_disk,load_dataset
import torch

c:\Users\Ankit Ghai\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-02-23 13:28:41,582: INFO: config: PyTorch version 2.4.1 available.]


In [12]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config


    def train(self):
        device="cuda" if torch.cuda.is_available() else "cpu"
        tokenizer=AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        ## Loading the dataset
        dataset_samsum_pt=load_from_disk(self.config.data_path)


        ## TRaining ARguments
        # trainer_args=TrainingArguments(
        # output_dir=self.config.root_dir,num_train_epochs=self.config.num_train_epochs,warmup_steps=self.config.warmup_steps,
        # per_device_train_batch_size=self.config.per_device_train_batch_size,per_device_eval_batch_size=self.config.per_device_eval_batch_size,
        # weight_decay=self.config.weight_decay,logging_steps=self.config.logging_steps,evaluation_strategy=self.config.evaluation_strategy,
        # eval_steps=self.config.eval_steps,save_steps=self.config.save_steps,gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # )


        trainer_args=TrainingArguments(
            output_dir=self.config.root_dir,num_train_epochs=1,warmup_steps=500,per_device_train_batch_size=1,
            per_device_eval_batch_size=1,weight_decay=0.01,logging_steps=10,
            evaluation_strategy='steps', eval_steps=500,save_steps=1e6,
            gradient_accumulation_steps=16
        )

        trainer=Trainer(model=model_pegasus,args=trainer_args,tokenizer=tokenizer,data_collator=seq2seq_data_collator,
                        train_dataset=dataset_samsum_pt["test"],eval_dataset=dataset_samsum_pt['validation'])
        


        trainer.train()

        ## Save model
        model_pegasus.save_pretrained("pegasus-samsum-model")

        ## Save Tokenizers

        tokenizer.save_pretrained("tokenizer")




In [ ]:
## Create the pipeline

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()

except Exception as e:
    raise e

[2025-02-23 13:28:41,845: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-23 13:28:41,848: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-23 13:28:41,849: INFO: common: created directory at : artifacts]
[2025-02-23 13:28:41,850: INFO: common: created directory at : artifacts/model_trainer]


c:\Users\Ankit Ghai\anaconda3\envs\textS\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ankit Ghai\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint

{'loss': 3.1101, 'grad_norm': 511.2713928222656, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [23:28<48:01, 92.97s/it]   

{'loss': 3.0467, 'grad_norm': 253.44041442871094, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

{'loss': 2.9904, 'grad_norm': 321.72283935546875, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}


 98%|█████████▊| 50/51 [1:50:12<01:12, 72.51s/it] 

{'loss': 2.8546, 'grad_norm': 733.7205810546875, 'learning_rate': 5e-06, 'epoch': 0.98}


100%|██████████| 51/51 [1:50:47<00:00, 61.19s/it]c:\Users\Ankit Ghai\anaconda3\envs\textS\lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
100%|██████████| 51/51 [1:52:49<00:00, 132.73s/it]


{'train_runtime': 6769.3209, 'train_samples_per_second': 0.121, 'train_steps_per_second': 0.008, 'train_loss': 3.0233642727720973, 'epoch': 1.0}
